# Best Practices for Using Runners

In [1]:
%pip uninstall -y todd_ai
%pip install --extra-index-url https://pypi.org/simple .. > /dev/null


Found existing installation: todd_ai 0.5.1
Uninstalling todd_ai-0.5.1:
  Successfully uninstalled todd_ai-0.5.1
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pathlib
import tempfile
import time
from pprint import pprint
from typing import Any, NoReturn, TypedDict

import torch
import torch.nn.functional as F
import torch.utils.data
from torch import nn

import todd
from todd.runners import Memo

## Preparation

### Models

In [3]:
@todd.registries.ModelRegistry.register_()
class RunnerModel(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self._weight = torch.nn.Parameter(torch.tensor(0.0))

    @property
    def weight(self) -> torch.nn.Parameter:
        return self._weight

    def _forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self._weight

    def forward(
        self,
        runner: todd.runners.BaseRunner,
        batch,
        memo: Memo,
        *args,
        **kwargs,
    ) -> Memo:
        log: dict[str, Any] | None = memo.get("log")
        y = self._forward(batch["x"])
        loss = F.l1_loss(y, batch["y"])
        memo["loss"] = loss
        if log is not None:
            log["batch"] = str(batch)
            log["weight"] = f"{self._weight.item():.3f}"
            log["loss"] = f"{loss:.3f}"
        return memo

### Datasets

In [4]:
class Sample(TypedDict):
    x: int
    y: int

In [5]:
@todd.registries.DatasetRegistry.register_()
class RunnerDataset(torch.utils.data.Dataset[int]):

    def __init__(self, n: int) -> None:
        self._data = list(range(1, n + 1))

    def __len__(self) -> int:
        return len(self._data)

    def __getitem__(self, index: int) -> Sample:
        x = self._data[index]
        return Sample(x=x, y=x * 2)

In [6]:
class Batch(TypedDict):
    x: torch.Tensor
    y: torch.Tensor

## Validators

In [7]:
config = todd.Config(
    type='Validator',
    name='validator',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    logger=dict(),
    callbacks=[],
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-08-10 09:10:12,418 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:12,419 11393:140704352767616][base.py:69 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpeb8wo07n
└── validator

2 directories, 0 files


In [8]:
config = todd.Config(
    type='Validator',
    name='validator',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[dict(type='LogCallback', interval=5)],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree $work_dirs


[2024-08-10 09:10:13,840 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:13,844 11393:140704352767616][base.py:69 todd.Validator.validator __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:13,855 11393:140704352767616][log.py:91 todd.Validator.validator after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-08-10 09:10:13,859 11393:140704352767616][log.py:91 todd.Validator.validator after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-08-10 09:10:13,864 11393:140704352767616][log.py:91 todd.Validator.validator after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-08-10 09:10:13,868 11393:140704352767616][log.py:91 todd.Validator.validator after_run_iter] INFO: Iter [20/20] batch={


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp9fnrnrsf
└── validator

2 directories, 0 files


## Trainers

### Iteration Based

In [9]:
config = todd.Config(
    type="IterBasedTrainer",
    name="iter_based_trainer",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-08-10 09:10:15,493 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:15,497 11393:140704352767616][base.py:69 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:15,497 11393:140704352767616][base.py:69 todd.IterBasedTrainer.iter_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:15,507 11393:140704352767616][log.py:91 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [1/8] batch={'x': tensor([2, 8]), 'y': tensor([ 4, 16])} weight=0.000 loss=10.000
[2024-08-10 09:10:15,510 11393:140704352767616][log.py:91 todd.IterBasedTrainer.iter_based_trainer after_run_iter] INFO: Iter [2/8] batch={'x': tensor([10,  6]), 'y': tensor([20, 12])} weight=0.000 loss=16.000
[2024-08-10 09:10:15,514 11393:140704352767616][log.py:91 todd.IterBasedTrainer.iter_based_

### Epoch Based

In [10]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="epoch_based_trainer",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[dict(type="LogCallback", interval=1)],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-08-10 09:10:15,564 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:15,570 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:15,570 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.epoch_based_trainer __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:15,576 11393:140704352767616][log.py:97 todd.EpochBasedTrainer.epoch_based_trainer before_run_epoch] INFO: Epoch [1/3]
[2024-08-10 09:10:15,592 11393:140704352767616][log.py:91 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [1/15] batch={'x': tensor([6, 4]), 'y': tensor([12,  8])} weight=0.000 loss=10.000
[2024-08-10 09:10:15,594 11393:140704352767616][log.py:91 todd.EpochBasedTrainer.epoch_based_trainer after_run_iter] INFO: Iter [2/15] batch={'x': tensor([5,

## Callbacks

### Log

In [11]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-08-10 09:10:15,660 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:16,644 11393:140704352767616][log.py:53 todd.Validator.log_callback bind] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.10.0
todd_version: 0.5.1
cuda_home: None
git_commit_id: 8cc8a4f
git_status: 
M todd/runners/callbacks/checkpoint.py
 M todd/runners/callbacks/tensorboard.py
 M tutorials/runners.ipynb
[2024-08-10 09:10:16,646 11393:140704352767616][base.py:69 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:16,650 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-08-10 09:10:

In [12]:
config = todd.Config(
    type='Validator',
    name='log_callback',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type='LogCallback',
            interval=5,
            with_file_handler=True,
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo
    !cat {work_dirs}/log_callback/*.log


[2024-08-10 09:10:16,676 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:16,679 11393:140704352767616][base.py:69 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:16,683 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-08-10 09:10:16,686 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-08-10 09:10:16,688 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-08-10 09:10:16,690 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter 


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpvo24ampe
└── log_callback
    └── 2024-08-10T09-10-16_678852-08-00.log

2 directories, 1 file

[2024-08-10 09:10:16,679 11393:140704352767616][base.py:69 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:16,683 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-08-10 09:10:16,686 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-08-10 09:10:16,688 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] batch={'x': tensor([15]), 'y': tensor([30])} weight=0.000 loss=30.000
[2024-08-10 09:10:16,690 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [20/20] batch={'x':

In [13]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="AverageETA"),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1)
    )
    runner.run()

[2024-08-10 09:10:19,619 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:19,627 11393:140704352767616][base.py:69 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:19,627 11393:140704352767616][base.py:69 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:20,149 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:01 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-08-10 09:10:20,680 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:01 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-08-10 09:10:21,201 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:0

In [14]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            eta=dict(type="EMA_ETA", ema=dict(decay=0.2)),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.strategy.module.register_forward_hook(
        lambda *args, **kwargs: time.sleep(0.1 * min(10, runner.iter_))
    )
    runner.run()

[2024-08-10 09:10:21,746 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:21,749 11393:140704352767616][base.py:69 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:21,749 11393:140704352767616][base.py:69 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:23,273 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:04 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-08-10 09:10:27,294 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [10/20] ETA 0:00:05 batch={'x': tensor([10]), 'y': tensor([20])} weight=0.000 loss=20.000
[2024-08-10 09:10:32,311 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [15/20] ETA 0:00:0

In [15]:
config = todd.Config(
    type="Validator",
    name="log_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(
            type="LogCallback",
            interval=5,
            collect_env=dict(verbose=False),
            with_file_handler=True,
            eta=dict(type="AverageETA"),
        ),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-08-10 09:10:37,352 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:37,586 11393:140704352767616][log.py:53 todd.Validator.log_callback bind] INFO: 
platform: macOS-14.0
nvidia_smi: None
python_version: 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]
pytorch_version: 2.0.1
torchvision_version: 0.15.2
opencv_version: 4.10.0
todd_version: 0.5.1
cuda_home: None
git_commit_id: 8cc8a4f
git_status: 
M todd/runners/callbacks/checkpoint.py
 M todd/runners/callbacks/tensorboard.py
 M tutorials/runners.ipynb
[2024-08-10 09:10:37,588 11393:140704352767616][base.py:69 todd.Validator.log_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:37,594 11393:140704352767616][log.py:91 todd.Validator.log_callback after_run_iter] INFO: Iter [5/20] ETA 0:00:00 batch={'x': tensor([5]), 'y': tensor([10])} weight=0.000 loss=10.000
[2024-

### Git

In [16]:
config = todd.Config(
    type="Validator",
    name="git_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(type="GitCallback", diff='HEAD -- ":(exclude)*.ipynb"'),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )

    !echo
    !cat {work_dirs}/git_callback/*.log

[2024-08-10 09:10:37,622 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:37,687 11393:140704352767616][git.py:49 todd.Validator.git_callback bind] INFO: Saving git diff to /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp_qzztl2r/git_callback/git_diff_2024-08-10T09-10-37_687220-08-00.log
[2024-08-10 09:10:37,691 11393:140704352767616][base.py:69 todd.Validator.git_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R



diff --git a/todd/runners/callbacks/checkpoint.py b/todd/runners/callbacks/checkpoint.py
index f5932b5..127e509 100644
--- a/todd/runners/callbacks/checkpoint.py
+++ b/todd/runners/callbacks/checkpoint.py
@@ -39,7 +39,7 @@ class CheckpointCallback(IntervalMixin[T], BaseCallback[T]):
     def bind(self, *args, **kwargs) -> None:
         super().bind(*args, **kwargs)
 
-        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)
+        self.work_dir.mkdir(parents=True, exist_ok=True)
 
         if self.runner.auto_resume and self.latest_checkpoint_dir.exists():
             load_from = self.latest_checkpoint_dir
@@ -59,15 +59,15 @@ class CheckpointCallback(IntervalMixin[T], BaseCallback[T]):
             self.runner.load_state_dict(state_dict, **self._load_state_dict)
 
     @property
-    def checkpoint_dir(self) -> pathlib.Path:
+    def work_dir(self) -> pathlib.Path:
         return self.runner.work_dir / 'checkpoints'
 
     @property
     def latest_checkpoint_dir(self) -> p

### Optimize

In [17]:
config = todd.Config(
    type="IterBasedTrainer",
    name="optimize_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

[2024-08-10 09:10:38,951 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:38,953 11393:140704352767616][base.py:69 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:38,954 11393:140704352767616][optimize.py:118 todd.IterBasedTrainer.optimize_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:38,954 11393:140704352767616][optimize.py:126 todd.IterBasedTrainer.optimize_callback before_run] DEBUG: Requires grad parameters
'_weight'
[2024-08-10 09:10:38,955 11393:140704352767616][optimize.py:130 todd.IterBasedTrainer.optimize_callback before_run] DEBUG: Total number of requires grad parameters: 1
[2024-08-10 09:10:38,953 11393:140704352767616][base.py:69 todd.IterBasedTrainer.optimize_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:38,954 11393:140704352767

### Learning Rate Schedule

In [18]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_schedule_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=5),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-08-10 09:10:38,996 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:38,998 11393:140704352767616][base.py:69 todd.IterBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:38,998 11393:140704352767616][optimize.py:118 todd.IterBasedTrainer.lr_schedule_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:38,999 11393:140704352767616][optimize.py:126 todd.IterBasedTrainer.lr_schedule_callback before_run] DEBUG: Requires grad parameters
'_weight'
[2024-08-10 09:10:39,000 11393:140704352767616][optimize.py:130 todd.IterBasedTrainer.lr_schedule_callback before_run] DEBUG: Total number of requires grad parameters: 1
[2024-08-10 09:10:39,003 11393:140704352767616][log.py:91 todd.IterBasedTrainer.lr_schedule_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([7, 5]), 'y': tensor([14, 10])} weight=

In [19]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="lr_schedule_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=4),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScheduleCallback",
            lr_scheduler=dict(type="LinearLR", total_iters=3),
            by_epoch=True,
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=5,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-08-10 09:10:39,031 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:39,032 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.lr_schedule_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:39,033 11393:140704352767616][optimize.py:118 todd.EpochBasedTrainer.lr_schedule_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:39,034 11393:140704352767616][optimize.py:126 todd.EpochBasedTrainer.lr_schedule_callback before_run] DEBUG: Requires grad parameters
'_weight'
[2024-08-10 09:10:39,034 11393:140704352767616][optimize.py:130 todd.EpochBasedTrainer.lr_schedule_callback before_run] DEBUG: Total number of requires grad parameters: 1
[2024-08-10 09:10:39,035 11393:140704352767616][log.py:97 todd.EpochBasedTrainer.lr_schedule_callback before_run_epoch] INFO: Epoch [1/5]
[2024-08-10 09:10:39,038 11393:140704352767616][lo

### Learning Rate Scale

In [20]:
config = todd.Config(
    type="IterBasedTrainer",
    name="lr_scale_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type="OptimizeCallback"),
        dict(
            type="LRScaleCallback",
            lr_scaler=dict(base_batch_size=1),
        ),
        dict(type="LogCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = todd.registries.RunnerRegistry.build(
        config,
        work_dir=dict(root=work_dirs),
    )
    runner.run()

[2024-08-10 09:10:39,142 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:39,144 11393:140704352767616][lr.py:95 todd.IterBasedTrainer.lr_scale_callback bind] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-08-10 09:10:39,145 11393:140704352767616][base.py:69 todd.IterBasedTrainer.lr_scale_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:39,146 11393:140704352767616][optimize.py:118 todd.IterBasedTrainer.lr_scale_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:39,146 11393:140704352767616][optimize.py:126 todd.IterBasedTrainer.lr_scale_callback before_run] DEBUG: Requires grad parameters
'_weight'
[2024-08-10 09:10:39,144 11393:140704352767616][lr.py:95 todd.IterBasedTrainer.lr_scale_callback bind] INFO: base_batch_size=1 batch_size=2 lr_scaler=2.000
[2024-08-10 09:10:39,145 11393:140704352767616][base.py:69 

### Checkpoint

In [21]:
config = todd.Config(
    type="IterBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    iters=8,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_5 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_5'
    for f in iter_5.glob('*.pth'):
        print(f'{f.name}:')
        pprint(torch.load(f, 'cpu'))
        print()

    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_5),
        )
    runner.run()


[2024-08-10 09:10:39,198 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:39,200 11393:140704352767616][base.py:69 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:39,201 11393:140704352767616][optimize.py:118 todd.IterBasedTrainer.checkpoint_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:39,202 11393:140704352767616][optimize.py:126 todd.IterBasedTrainer.checkpoint_callback before_run] DEBUG: Requires grad parameters
'_weight'
[2024-08-10 09:10:39,203 11393:140704352767616][optimize.py:130 todd.IterBasedTrainer.checkpoint_callback before_run] DEBUG: Total number of requires grad parameters: 1
[2024-08-10 09:10:39,205 11393:140704352767616][log.py:91 todd.IterBasedTrainer.checkpoint_callback after_run_iter] INFO: Iter [1/8] batch={'x': tensor([3, 7]), 'y': tensor([ 6, 14])} weight=0.000


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpm_guz5jx
└── checkpoint_callback
    └── checkpoints
        ├── iter_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_5
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── opt

[2024-08-10 09:10:41,062 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:41,064 11393:140704352767616][checkpoint.py:54 todd.IterBasedTrainer.checkpoint_callback bind] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpm_guz5jx/checkpoint_callback/checkpoints/iter_5
[2024-08-10 09:10:41,067 11393:140704352767616][base.py:69 todd.IterBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-08-10 09:10:41,069 11393:140704352767616][base.py:69 todd.IterBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:41,070 11393:140704352767616][optimize.py:118 todd.IterBasedTrainer.checkpoint_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:41,070 11393:140704352767616][optimize.py:126 todd.IterBasedTrainer.checkpoint_callback before_run] DEBUG: Requir

strategy.pth:
{}

optim.pth:
{'param_groups': [{'dampening': 0,
                   'differentiable': False,
                   'foreach': None,
                   'lr': 0.005,
                   'maximize': False,
                   'momentum': 0,
                   'nesterov': False,
                   'params': [0],
                   'weight_decay': 0}],
 'state': {0: {'momentum_buffer': None}}}

meta.pth:
{'iter_': 5}

model.pth:
OrderedDict([('_weight', tensor(0.1375))])

callbacks.pth:
{'callbacks': [{}, {}, {}]}



In [22]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=2),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    iter_8 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_8'
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_8),
        )
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    iter_10 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'iter_10'
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(iter_10),
        )
    runner.run()


[2024-08-10 09:10:41,148 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:41,150 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:41,151 11393:140704352767616][optimize.py:118 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:41,151 11393:140704352767616][optimize.py:126 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: Requires grad parameters
'_weight'
[2024-08-10 09:10:41,152 11393:140704352767616][optimize.py:130 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: Total number of requires grad parameters: 1
[2024-08-10 09:10:41,153 11393:140704352767616][log.py:97 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-08-10 09:10:41,156 11393:140704352767616][log.py:


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpszivugje
└── checkpoint_callback
    └── checkpoints
        ├── iter_10
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_12
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_14
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_4
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── iter_6
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── 

[2024-08-10 09:10:42,981 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:42,983 11393:140704352767616][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback bind] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpszivugje/checkpoint_callback/checkpoints/iter_8
[2024-08-10 09:10:42,987 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-08-10 09:10:42,988 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:42,989 11393:140704352767616][optimize.py:118 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:42,990 11393:140704352767616][optimize.py:126 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: R


--------------------



[2024-08-10 09:10:44,856 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:44,858 11393:140704352767616][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback bind] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpszivugje/checkpoint_callback/checkpoints/iter_10
[2024-08-10 09:10:44,862 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-08-10 09:10:44,863 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:44,864 11393:140704352767616][optimize.py:118 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:44,864 11393:140704352767616][optimize.py:126 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: 

In [23]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="checkpoint_callback",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !tree {work_dirs}
    !echo

    epoch_2 = pathlib.Path(work_dirs) / 'checkpoint_callback' / 'checkpoints' / 'epoch_2'
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(
            config,
            work_dir=dict(root=work_dirs),
            load_from=str(epoch_2),
        )
    runner.run()


[2024-08-10 09:10:44,907 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:44,909 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:44,910 11393:140704352767616][optimize.py:118 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:44,910 11393:140704352767616][optimize.py:126 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: Requires grad parameters
'_weight'
[2024-08-10 09:10:44,911 11393:140704352767616][optimize.py:130 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: Total number of requires grad parameters: 1
[2024-08-10 09:10:44,911 11393:140704352767616][log.py:97 todd.EpochBasedTrainer.checkpoint_callback before_run_epoch] INFO: Epoch [1/3]
[2024-08-10 09:10:44,914 11393:140704352767616][log.py:


/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp8v6lb36t
└── checkpoint_callback
    └── checkpoints
        ├── epoch_1
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_2
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        ├── epoch_3
        │   ├── callbacks.pth
        │   ├── meta.pth
        │   ├── model.pth
        │   ├── optim.pth
        │   └── strategy.pth
        └── latest -> /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp8v6lb36t/checkpoint_callback/checkpoints/epoch_3

7 directories, 15 files



[2024-08-10 09:10:46,765 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:46,767 11393:140704352767616][checkpoint.py:54 todd.EpochBasedTrainer.checkpoint_callback bind] INFO: Loading from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmp8v6lb36t/checkpoint_callback/checkpoints/epoch_2
[2024-08-10 09:10:46,771 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.checkpoint_callback load_model_state_dict] INFO: <All keys matched successfully>
[2024-08-10 09:10:46,772 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.checkpoint_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:46,773 11393:140704352767616][optimize.py:118 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: Training modules
''
[2024-08-10 09:10:46,774 11393:140704352767616][optimize.py:126 todd.EpochBasedTrainer.checkpoint_callback before_run] DEBUG: 

### Monitor

In [24]:
class CustomError(RuntimeError):
    pass

In [25]:
@todd.registries.RunnerRegistry.register_()
class FaultyValidator(todd.runners.Validator):

    def _run_iter(self, *args, **kwargs) -> NoReturn:
        raise CustomError("faulty runner")

In [26]:
config = todd.Config(
    type='FaultyValidator',
    name='monitor_callback',
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=20),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=1),
    callbacks=[
        dict(type='MonitorCallback'),
        dict(type='LogCallback', interval=5, with_file_handler=True),
    ],
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    try:
        runner.run()
    except CustomError as e:
        pass

    !echo
    !cat {work_dirs}/monitor_callback/*.log


[2024-08-10 09:10:46,820 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:46,822 11393:140704352767616][base.py:69 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:46,824 11393:140704352767616][monitor.py:31 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x10b33ea10>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 295, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_11393/454645826.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: 


[2024-08-10 09:10:46,822 11393:140704352767616][base.py:69 todd.FaultyValidator.monitor_callback __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:46,824 11393:140704352767616][monitor.py:31 todd.FaultyValidator.monitor_callback __exit__] ERROR: Unable to run iter_=1
batch={'x': tensor([1]), 'y': tensor([2])}
memo={'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x10b33ea10>}
Traceback (most recent call last):
  File "/Users/bytedance/.local/share/virtualenvs/todd-ARrcnwyq/lib/python3.11/site-packages/todd/runners/base.py", line 295, in _run
    memo = self._run_iter(batch, memo)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/ipykernel_11393/454645826.py", line 5, in _run_iter
    raise CustomError("faulty runner")
CustomError: faulty runner


### Priorities

## Strategies

In [27]:
config = todd.Config(
    type="EpochBasedTrainer",
    name="strategy_load_model_from",
    strategy=dict(type='BaseStrategy'),
    dataset=dict(type='RunnerDataset', n=10),
    model=dict(type='RunnerModel'),
    dataloader=dict(batch_size=2, shuffle=True),
    callbacks=[
        dict(type='OptimizeCallback'),
        dict(type='LogCallback', interval=1),
        dict(type="CheckpointCallback", interval=1, by_epoch=True),
    ],
    optimizer=dict(type="SGD", lr=0.005),
    epochs=3,
    logger=dict(),
)
with tempfile.TemporaryDirectory() as work_dirs:
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.run()

    !echo
    !echo {'-' * 20}
    !echo

    epoch_2 = (pathlib.Path(work_dirs) / 'strategy_load_model_from' / 'checkpoints' / 'epoch_2' / 'model.pth')
    runner: todd.runners.BaseRunner = \
        todd.registries.RunnerRegistry.build(config, work_dir=dict(root=work_dirs))
    runner.strategy.load_model_from(epoch_2)
    runner.run()


[2024-08-10 09:10:48,072 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:48,074 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:48,075 11393:140704352767616][optimize.py:118 todd.EpochBasedTrainer.strategy_load_model_from before_run] DEBUG: Training modules
''
[2024-08-10 09:10:48,076 11393:140704352767616][optimize.py:126 todd.EpochBasedTrainer.strategy_load_model_from before_run] DEBUG: Requires grad parameters
'_weight'
[2024-08-10 09:10:48,076 11393:140704352767616][optimize.py:130 todd.EpochBasedTrainer.strategy_load_model_from before_run] DEBUG: Total number of requires grad parameters: 1
[2024-08-10 09:10:48,077 11393:140704352767616][log.py:97 todd.EpochBasedTrainer.strategy_load_model_from before_run_epoch] INFO: Epoch [1/3]
[2024-08-10 09:10:48,080 11393


--------------------



[2024-08-10 09:10:49,897 11393:140704352767616][patches.py:214 todd dataloader_build_pre_hook] INFO: `worker_init_fn` is recommended to be 'default', instead of None.
[2024-08-10 09:10:49,900 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.strategy_load_model_from __init__] DEBUG: Rank 0 initialized by bytedance@C02G870SMD6R
[2024-08-10 09:10:49,901 11393:140704352767616][base.py:84 todd.EpochBasedTrainer.strategy_load_model_from load_model_from] INFO: Loading model from /var/folders/v_/1kkfntxs5z74_rwvy1f3_mp80000gn/T/tmpdtuy90xh/strategy_load_model_from/checkpoints/epoch_2/model.pth
[2024-08-10 09:10:49,903 11393:140704352767616][base.py:69 todd.EpochBasedTrainer.strategy_load_model_from load_model_state_dict] INFO: <All keys matched successfully>
[2024-08-10 09:10:49,904 11393:140704352767616][optimize.py:118 todd.EpochBasedTrainer.strategy_load_model_from before_run] DEBUG: Training modules
''
[2024-08-10 09:10:49,905 11393:140704352767616][optimize.py:126 todd.EpochBasedT

## Dry Run

In [28]:
todd.Store.DRY_RUN = True